<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Gmail - Create draft email

**Tags:** #gmail #email #draft #create #python #library

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook will show how to create a draft email using the Gmail API. It is usefull for organizations that need to automate the creation of emails.

**References:**
- [Gmail API Documentation](https://developers.google.com/gmail/api/quickstart/python)
- [Gmail API Python Quickstart](https://developers.google.com/gmail/api/quickstart/python)

## Input

### Import libraries

In [ ]:
import base64
import email
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

### Setup Variables
- `SCOPES`: list of scopes to be used for the Gmail API
- `credentials`: credentials to be used for the Gmail API

For more information on how to setup the credentials, please refer to the [Gmail API Python Quickstart](https://developers.google.com/gmail/api/quickstart/python).

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/gmail.compose"]
credentials = None

## Model

### Create draft email

This function will create a draft email using the Gmail API.

In [ ]:
def create_draft_email(service, user_id, message_body):
    try:
        message = {"message": message_body}
        draft = service.users().drafts().create(userId=user_id, body=message).execute()
        print("Draft email created with id: %s" % draft["id"])
        return draft
    except Exception as e:
        print("An error occurred: %s" % e)

## Output

### Display result

In [ ]:
# Create a draft email
create_draft_email(service, user_id, message_body)